In [38]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [39]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [40]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [41]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [42]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [43]:
prompt_uc = """
Purchase
Description
This feature allows users to purchase of items they have added to their shopping cart or at product detail page.
Actor: User
Preconditions: None
Postconditions: user can checkout order.
Basic Flow:
Step 1: User in the shopping cart page and User has added items to the shopping cart.
Step 2: User select items to checkout by selecting each items by clicking on checkbox before item.
Step 3: System displays the summary costs.
Step 4: User click the "Checkout" button.
Step 5: System process to checkout.
- User is redirected to page that shows new orders of all the items user have chosen.
- For each product, there should be the information section of the product including name, options, quantity, price, which shop sells it, promotion information, the shipping information section, the platform's promotion information section and the payment section with the default payment method.
- In the order there should be default delivery location chosen by the system for user.
- Products from different shops will be separate into different orders, products from the same shop is in one order. 

Alterative flow: 
1. In shopping cart page: User can select all the items come from one store, user choose by clicking the checkbox at the head of the store.
2. User can purchase products in product detail page:
- User perform feature 'View a product detail'.
- User click button 'Buy Now'.
- If product has many options, user choose one available option before adding product to cart. 
- User can adjust quantity of product before adding product into cart by clicking on minus or plus button next to quantity of the product. 
- User click the "Checkout" button.
- System process to checkout.


Exception flow:
- user cannot click on the checkbox of the product that is out of stock or erased by the seller even though it is in user's shopping cart.
When user purchase products in product detail page:
- user cannot purchase product has many options without selecting one available option.
- user cannot purchase product with quantity that is bigger than the current stock or lower than one.
- User cannot purchase product with no stock or out of stock option of one product with multiple options.
- the button "Checkout" is disable if the selected product is invalid 
"""

In [44]:
use_case_directory_path = r"D:\GPT-testing\SpecificationData\MatchaEnglishWebsite"
test_scenario_directory_path = r"D:\Dissertation-GPT\evaluate\baseline 2\matcha\ts"
save_testcase_path = r"D:\Dissertation-GPT\evaluate\baseline 2\book\tc"
save_testcase_ref_path = r"D:\Dissertation-GPT\evaluate\baseline 2\book\tc refined"

In [45]:
def parse_test_cases(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    
    test_cases = content.strip().split('\n\n')
    
    parsed_test_cases = {}
    test_case_list = []
    
    for test_case in test_cases:
        lines = test_case.split('\n')
        test_case_dict = {}
        steps = []
        
        for line in lines:
            if line.startswith('Test Case:'):
                test_case_dict['testCase'] = line.split(': ')[1]
            elif line.startswith('testCaseName:'):
                test_case_dict['testCaseName'] = line.split(': ')[1]
            elif line.startswith('objective:'):
                test_case_dict['objective'] = line.split(': ')[1]
            elif line.startswith('  - Step'):
                steps.append(line.split(': ')[1])
            elif line.startswith('expectedResult:'):
                test_case_dict['expectedResult'] = line.split(': ')[1]
        
        test_case_dict['testSteps'] = steps
        test_case_list.append(test_case_dict)
    
    parsed_test_cases['testCases'] = test_case_list
    
    return json.dumps(parsed_test_cases, indent=4)

In [46]:
use_case_list = [
"Remove the book.txt",
"Display book details.txt",
"Display genre details.txt",
"Display the list of authors.txt",
"Display the list of books.txt",
"Display the list of genres.txt",
"Edit the author.txt",
"Edit the book.txt",
"Edit the genre.txt",
"Remove the author.txt",
"Remove the genre.txt",
"Add a new author.txt",
"Add a new book.txt",
"Add a new genre.txt",
"Display author details.txt",
]

In [47]:

for filename in os.listdir(save_testcase_path):
    usecasename = filename.split("-")[0]
    testscenario = filename.split(".txt")[0]
    testscenario = testscenario.split("-")[1]
    print(usecasename)
    print(testscenario)

    file_path = os.path.join(save_testcase_path, filename)
    testcases = parse_test_cases(file_path)

    content = "Test scenarios:"+ testscenario+"\n" + testcases
    promptSelfRef = [
                { "role": "system", "content": SELF_REF},
                { "role": "user", "content": content }
            ]
    final_response = ask(promptSelfRef, client, model)
    testcase_path = os.path.join (save_testcase_ref_path,f"{usecasename}-{testscenario}.txt" )
    write_json_to_txt(final_response,testcase_path )


Add a new author
10. VerifyNavigationAwayDoesNotSaveData


KeyboardInterrupt: 